In [ ]:
import pandas as pd

In [ ]:
!git clone https://github.com/ysut/2024_SplicingSNVScreening_test

In [ ]:
import os

# Move to the working directory
path = '/content/2024_SplicingSNVScreening_test/analysis'
os.chdir(path)
os.listdir()

In [ ]:
from libs import utils, preprocess, variantfilter, posparser, splaiparser
from libs import predeffect, scoring
from libs import anno_spliceai, anno_clinvar
from libs.deco import print_filtering_count